# 04. Mini Time Series

This notebook implements a minimal LSTM for time series prediction using synthetic data.

## Experiment Overview
- **Goal**: Predict future values in synthetic time series data
- **Model**: LSTM-based sequence predictor
- **Features**: Sine wave prediction, trend analysis, multiple forecasting horizons
- **Learning**: Understanding time series forecasting with neural networks

## What You'll Learn
- Time series data preparation
- LSTM for sequence prediction
- Multiple forecasting horizons
- Trend and seasonality modeling



In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add scripts directory to path
sys.path.append('../scripts')
from utils import get_device, set_seed, create_time_series_data
from train import train_model

# Set random seed for reproducibility
set_seed(42)

# Get device
device = get_device()
print(f"Using device: {device}")

# Generate synthetic time series data
print("Generating synthetic time series data...")
n_samples = 2000
time_series = create_time_series_data(n_samples=n_samples, n_features=1, noise=0.1, trend=0.01, seasonality=True)

# Plot the time series
plt.figure(figsize=(12, 4))
plt.plot(time_series[:500])
plt.title('Synthetic Time Series Data (First 500 points)')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()

print(f"Time series shape: {time_series.shape}")
print(f"Data range: [{time_series.min():.3f}, {time_series.max():.3f}]")


In [ ]:
# Define the Time Series LSTM model
class TimeSeriesLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1):
        super(TimeSeriesLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        
        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))
        
        # Take the last output
        out = self.fc(out[:, -1, :])
        
        return out

# Create model instance
model = TimeSeriesLSTM().to(device)

# Print model summary
print("Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Model size: {sum(p.numel() for p in model.parameters()) * 4 / 1024 / 1024:.2f} MB")


In [ ]:
# Prepare time series data for training
def create_sequences(data, seq_length=60, forecast_horizon=1):
    """Create input-output sequences for time series prediction."""
    X, y = [], []
    
    for i in range(len(data) - seq_length - forecast_horizon + 1):
        # Input sequence
        X.append(data[i:i + seq_length])
        # Target (next value(s))
        y.append(data[i + seq_length:i + seq_length + forecast_horizon])
    
    return np.array(X), np.array(y)

# Create sequences
seq_length = 60
forecast_horizon = 1
X, y = create_sequences(time_series, seq_length, forecast_horizon)

print(f"Number of sequences: {len(X)}")
print(f"Sequence length: {seq_length}")
print(f"Forecast horizon: {forecast_horizon}")
print(f"Input shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split data
train_size = int(0.8 * len(X))
val_size = int(0.1 * len(X))

X_train = X[:train_size]
y_train = y[:train_size]
X_val = X[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]
X_test = X[train_size + val_size:]
y_test = y[train_size + val_size:]

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train).unsqueeze(-1)  # Add feature dimension
y_train = torch.FloatTensor(y_train)
X_val = torch.FloatTensor(X_val).unsqueeze(-1)
y_val = torch.FloatTensor(y_val)
X_test = torch.FloatTensor(X_test).unsqueeze(-1)
y_test = torch.FloatTensor(y_test)

print(f"\nTraining set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")


In [ ]:
# Custom training function for time series
def train_time_series(model, X_train, y_train, X_val, y_val, epochs=50, lr=0.001, device='cpu'):
    """Train time series model."""
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        for i in range(0, len(X_train), 32):  # Batch size 32
            batch_X = X_train[i:i+32].to(device)
            batch_y = y_train[i:i+32].to(device)
            
            optimizer.zero_grad()
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i in range(0, len(X_val), 32):
                batch_X = X_val[i:i+32].to(device)
                batch_y = y_val[i:i+32].to(device)
                output = model(batch_X)
                loss = criterion(output, batch_y)
                val_loss += loss.item()
        
        train_loss /= (len(X_train) // 32)
        val_loss /= (len(X_val) // 32)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')
    
    return train_losses, val_losses

# Train the model
print("Starting time series training...")
train_losses, val_losses = train_time_series(model, X_train, y_train, X_val, y_val, epochs=50, device=device)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Time Series Training History')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training History (Log Scale)')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss (log)')
plt.yscale('log')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('../results/plots/time_series_training.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Evaluate the model and make predictions
model.eval()
with torch.no_grad():
    # Test predictions
    test_predictions = model(X_test.to(device)).cpu().numpy()
    
    # Calculate metrics
    mse = np.mean((test_predictions - y_test.numpy()) ** 2)
    mae = np.mean(np.abs(test_predictions - y_test.numpy()))
    rmse = np.sqrt(mse)
    
    print(f"Test Metrics:")
    print(f"MSE: {mse:.6f}")
    print(f"MAE: {mae:.6f}")
    print(f"RMSE: {rmse:.6f}")

# Visualize predictions
plt.figure(figsize=(15, 8))

# Plot 1: Test predictions vs actual
plt.subplot(2, 2, 1)
test_indices = range(len(test_predictions))
plt.plot(test_indices, y_test.numpy()[:100], label='Actual', alpha=0.7)
plt.plot(test_indices, test_predictions[:100], label='Predicted', alpha=0.7)
plt.title('Test Predictions vs Actual (First 100 points)')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True)

# Plot 2: Scatter plot
plt.subplot(2, 2, 2)
plt.scatter(y_test.numpy(), test_predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Predicted vs Actual Scatter')
plt.grid(True)

# Plot 3: Residuals
plt.subplot(2, 2, 3)
residuals = test_predictions - y_test.numpy()
plt.plot(test_indices, residuals[:100])
plt.title('Residuals (First 100 points)')
plt.xlabel('Time Step')
plt.ylabel('Residual')
plt.grid(True)

# Plot 4: Full time series with predictions
plt.subplot(2, 2, 4)
# Show last part of training + test predictions
train_end = len(time_series) - len(test_predictions)
plt.plot(range(train_end, len(time_series)), time_series[train_end:], label='Actual', alpha=0.7)
plt.plot(range(train_end + seq_length, len(time_series)), test_predictions, label='Predicted', alpha=0.7)
plt.title('Full Time Series with Predictions')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('../results/plots/time_series_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

# Multi-step ahead prediction
def predict_future(model, last_sequence, steps=20):
    """Predict future values using the model."""
    model.eval()
    predictions = []
    current_seq = last_sequence.clone()
    
    with torch.no_grad():
        for _ in range(steps):
            pred = model(current_seq.unsqueeze(0).to(device))
            predictions.append(pred.cpu().item())
            
            # Update sequence: remove first element, add prediction
            current_seq = torch.cat([current_seq[1:], pred], dim=0)
    
    return predictions

# Make future predictions
future_steps = 50
last_seq = X_test[-1]  # Last sequence from test set
future_predictions = predict_future(model, last_seq, future_steps)

# Plot future predictions
plt.figure(figsize=(12, 6))
# Show last part of actual data
start_idx = len(time_series) - 200
plt.plot(range(start_idx, len(time_series)), time_series[start_idx:], label='Actual', color='blue')
# Show future predictions
future_indices = range(len(time_series), len(time_series) + future_steps)
plt.plot(future_indices, future_predictions, label='Future Predictions', color='red', linestyle='--')
plt.title('Future Time Series Predictions')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.savefig('../results/plots/time_series_future.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Generated {future_steps} future predictions")
print(f"Future prediction range: [{min(future_predictions):.3f}, {max(future_predictions):.3f}]")
